# Bike Count Data Extraction Notebook

The raw bike count data was downloaded from Open Data Paris [(comptage vélo)](https://opendata.paris.fr/explore/dataset/comptage-velo-historique-donnees-compteurs/information) and covers the period from 2019 to 2024.

The dataset includes measurements from dozens of bike counting sites across Paris. For the current analysis, I selected five sites that serve as reasonable proxies for recreational cycling routes that I personally use.

This notebook focuses on the steps required to extract, filter, and concatenate data from these five sites.

In [1]:
# libraries
import sys
from pathlib import Path
import pandas as pd

# define directory variables
REPO_ROOT = Path.cwd().parent
DATA_DIR = REPO_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
DATA_DIR.mkdir(exist_ok=True)

### Data Quality & Structural Notes

- Date/time formatting was inconsistent across years and was normalized to a single format during preprocessing.

- The pyarrow engine had difficulty parsing the 2024 data file, requiring additional steps to ensure correct site_id values.

- The number of counters per site varies over time (ranging from 1 to 4), which has implications for aggregation and longitudinal comparisons.

- Each site records traffic in both road directions.

- At the Pont du Garigliano site, two distinct site_id values correspond to traffic flowing in opposite directions.

In [2]:
# read in function
%run ../scripts/normalize_datetime_hourly.py

# raw file names
raw_files_list = [ '2019_comptage-velo-donnees-compteurs.csv',
                   '2020_comptage-velo-donnees-compteurs.csv',
                   '2021_comptage-velo-donnees-compteurs.csv',
                   '2022_comptage-velo-donnees-compteurs.csv',
                   '2023_comptage-velo-donnees-compteurs.csv',
                   '2024_comptage-velo-donnees-compteurs.csv']

# columns of interest
cols = ['Date et heure de comptage', 
        'Nom du site de comptage', 
        'Nom du compteur',
        'Identifiant du site de comptage', 
        'Identifiant du compteur',
        'Comptage horaire', 
        'Coordonnées géographiques']

# read in raw data
dfs = []
for file in raw_files_list:
    df = pd.read_csv(
        RAW_DIR / file,
        sep=";",
        engine= "pyarrow", # ----- Use "python" engine while troubleshooting 
        usecols=cols,
        #nrows = 100, # -----Uncomment while trouble shooting to speed up read time; does not work with "pyarrow" 
        dtype={
            'Nom du site de comptage': 'string', 
            'Nom du compteur': 'string', 
            'Identifiant du site de comptage': 'string',
            'Identifiant du compteur': 'string',
            'Comptage horaire': 'Int64', 
            'Coordonnées géographiques' : 'object'
        }
    )
    # handle unexpected .0 suffixes that crop up in 2024 file, probably a pyarrow parse issue
    sid = (df["Identifiant du site de comptage"]
          .astype("string")
          .str.replace("\u00a0", " ", regex=False)  # NBSP safety
          .str.strip()
    )
    df["Identifiant du site de comptage"] = (pd.to_numeric(sid, errors="coerce")
                                            .astype("Int64")     
                                            .astype("string")    
    )
    df["year"] = file[:4]
    df = normalize_datetime_hourly(
                                   df,
                                   "Date et heure de comptage",
                                   ambiguous="NaT",
                                   resolve_ambiguous="second")
    dfs.append(df)

main_df = pd.concat(dfs, axis=0, ignore_index=True)

# filter to sites of interest
sites_of_interest = ['100047535', '100056226', '100047551', '100047542', '100047547', '100047546']
main_df = main_df[ main_df['Identifiant du site de comptage'].isin(sites_of_interest)]

# rename variables
main_df.rename(columns={'Identifiant du compteur' : 'counter_id',
                        'Nom du compteur' : 'counter_name',
                        'Identifiant du site de comptage' : 'site_id',
                        'Nom du site de comptage' : 'site_name_raw',
                        'Comptage horaire' : 'count', 
                        'Date et heure de comptage': 'date_time_raw',
                        'Coordonnées géographiques': 'coords',
                        'Date et heure de comptage_dt' : 'date_time_dt'}, 
              inplace = True)

# select and reorder columns
main_df = main_df[ ['year', 'date_time_dt', 'site_id', 'site_name_raw',  'counter_id', 'counter_name',  'count', 'coords']]

# convert names to lowercase (to normalize capitalization aross years)
main_df['site_name'] = main_df['site_name_raw'].str.lower()
main_df['counter_name'] = main_df['counter_name'].str.lower()

# simplify site_name to describe loaciton in a general way
main_df['site_name'] = (
    main_df['site_name']
    .str.replace("no-se|se-no", "", regex = True)
    .str.replace("face au 8 ","", regex = True)
    .str.replace("face au 48 ","", regex = True)
    .str.replace("face au 70 ","", regex = True)
    .str.replace("^6 ","", regex = True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# select and reorder columns
main_df = main_df[ ['year', 'date_time_dt', 'site_id', 'site_name_raw', 'site_name',  'counter_id', 'counter_name',  'count', 'coords']]

# display
print(main_df.dtypes)
display(main_df)

year                                   object
date_time_dt     datetime64[ns, Europe/Paris]
site_id                        string[python]
site_name_raw                  string[python]
site_name                      string[python]
counter_id                     string[python]
counter_name                   string[python]
count                                   Int64
coords                                 object
dtype: object


,year,date_time_dt,site_id,site_name_raw,site_name,counter_id,counter_name,count,coords
452,2019,2019-01-01 07:00:00+01:00,100047535,Pont du Garigliano NO-SE,pont du garigliano,100047535-SC,pont du garigliano no-se,3,"48.839927,2.267151"
453,2019,2019-01-04 02:00:00+01:00,100047535,Pont du Garigliano NO-SE,pont du garigliano,100047535-SC,pont du garigliano no-se,1,"48.839927,2.267151"
454,2019,2019-01-07 06:00:00+01:00,100047535,Pont du Garigliano NO-SE,pont du garigliano,100047535-SC,pont du garigliano no-se,13,"48.839927,2.267151"
455,2019,2019-01-07 07:00:00+01:00,100047535,Pont du Garigliano NO-SE,pont du garigliano,100047535-SC,pont du garigliano no-se,55,"48.839927,2.267151"
456,2019,2019-01-07 08:00:00+01:00,100047535,Pont du Garigliano NO-SE,pont du garigliano,100047535-SC,pont du garigliano no-se,114,"48.839927,2.267151"
...,...,...,...,...,...,...,...,...,...
6521217,2024,2024-12-20 10:00:00+01:00,100047546,Face au 70 quai de Bercy,quai de bercy,100047546-103047546,face au 70 quai de bercy s-n,70,"48.8295233, 2.38699"
6521218,2024,2024-12-24 04:00:00+01:00,100047546,Face au 70 quai de Bercy,quai de bercy,100047546-103047546,face au 70 quai de bercy s-n,0,"48.8295233, 2.38699"
6521219,2024,2024-12-25 02:00:00+01:00,100047546,Face au 70 quai de Bercy,quai de bercy,100047546-103047546,face au 70 quai de bercy s-n,0,"48.8295233, 2.38699"
6521220,2024,2024-12-29 05:00:00+01:00,100047546,Face au 70 quai de Bercy,quai de bercy,100047546-103047546,face au 70 quai de bercy s-n,4,"48.8295233, 2.38699"


In [3]:
# Optional: save the processed dataset for reuse
# This step is disabled by default to avoid unnecessary file writes
# Uncomment to write the data to disk
main_df.to_parquet(DATA_DIR / "extracted/extracted_hourly_bike_count_data.parquet")